In [13]:
import polars as pl
import os
from typing import Dict
current_path = "/Users/febinsathar/codebase/integration-transformation-layer/kandui"


In [2]:
from framework.business.compare.config.dto import TemplateRunTime,AggregationOperator
from framework import TemplateType
from framework.business.compare.config import  get_template_config_loader

init logger
init InProcessBatch


In [3]:
def check_file_existence(self, file_description):
    file_exists = os.path.exists(self)
    if file_exists:
        print(f'Found {file_description}')
        return file_exists, self
    else:
        print(f'{file_description} is missing')
        return file_exists, None

In [4]:
template_run_time = TemplateRunTime()

In [5]:
template_run_time.config = get_template_config_loader(TemplateType.PURCHASE)

In [6]:
paths = template_run_time.config.get_file_path()
template_run_time.valid, template_run_time.customer_path = check_file_existence(
    os.path.join(current_path, f'input/{paths.customer_file}'), paths.customer_file)

if template_run_time.valid:
    template_run_time.valid, template_run_time.elt_output_path = check_file_existence(
        os.path.join(current_path, f'output/{paths.elt_output_file}'), paths.elt_output_file)
    template_run_time.output_path = os.path.join(current_path, f'output/{paths.output_file}')
else:
    logger.error("customer_file missing")

Found customer-pr.csv
Found purchase_register.csv


In [7]:
df_customer = pl.read_csv(template_run_time.customer_path)
df_output = pl.read_csv(template_run_time.elt_output_path)

In [8]:
template_config = template_run_time.config

In [14]:
def roll_up_to_doc_level(df_customer: pl.dataframe, df_output: pl.dataframe,
                         doc_level_aggregator: Dict[str, AggregationOperator]):
    agg_expr = []
    for col, agg_func in doc_level_aggregator.items():
        if agg_func == AggregationOperator.SUM:
            agg_expr.append(pl.col(col).sum().alias(col))
        elif agg_func == AggregationOperator.FIRST:
            agg_expr.append(pl.col(col).first().alias(col))
    df_output_doc_level = df_output.group_by("Composite_Key").agg(agg_expr)
    df_customer_doc_level = df_customer.group_by("Composite_Key").agg(agg_expr)
    return df_output_doc_level, df_customer_doc_level

In [15]:
composite_key_columns = template_config.composite_keys()
composite_key_expr = pl.concat_str(
    [pl.col(col).fill_null("").cast(pl.Utf8) for col in composite_key_columns]
)
df_customer = df_customer.with_columns(composite_key_expr.alias("Composite_Key"))
df_output = df_output.with_columns(composite_key_expr.alias("Composite_Key"))
df_output_doc_level, df_customer_doc_level = roll_up_to_doc_level(df_customer, df_output,
                                                                  template_config.doc_level_aggregator())

df_customer_doc_level = df_customer_doc_level.with_columns(pl.col("Composite_Key").alias("Composite_Key_Cus"))
df_output_doc_level = df_output_doc_level.with_columns(pl.col("Composite_Key").alias("Composite_Key_Out"))


In [17]:
df_output_doc_level

Composite_Key,Voucher Number,My GSTIN,Supplier GSTIN,Item Taxable Value *,Composite_Key_Out
str,i64,str,str,f64,str
"""423540002226AA…",4235400022,"""26AADCK8554D1Z…",null,6175.0,"""423540002226AA…"
"""423120013626AA…",4231200136,"""26AADCK8554D1Z…","""24AAVFR2508P1Z…",10015.71,"""423120013626AA…"
"""423120027026AA…",4231200270,"""26AADCK8554D1Z…","""24AKHPC3335R1Z…",520.0,"""423120027026AA…"
"""423110003726AA…",4231100037,"""26AADCK8554D1Z…","""27AAAFQ4771C1Z…",16200.0,"""423110003726AA…"
"""423250001626AA…",4232500016,"""26AADCK8554D1Z…","""27AABCG0609F1Z…",28398.16,"""423250001626AA…"
"""423250010026AA…",4232500100,"""26AADCK8554D1Z…","""27AAACU1299G1Z…",1700.0,"""423250010026AA…"
"""423140001926AA…",4231400019,"""26AADCK8554D1Z…","""24DRFPS7605C1Z…",48900.0,"""423140001926AA…"
"""423110031726AA…",4231100317,"""26AADCK8554D1Z…","""24AUAPA4309A1Z…",1.65495e6,"""423110031726AA…"
"""423110056326AA…",4231100563,"""26AADCK8554D1Z…","""27AAHCA8513A1Z…",1.62525e6,"""423110056326AA…"


In [18]:
joined_df = df_output_doc_level.join(
    df_customer_doc_level,
    on="Composite_Key",
    how="outer",
    suffix="_cus"
)

In [19]:
joined_df


Composite_Key,Voucher Number,My GSTIN,Supplier GSTIN,Item Taxable Value *,Composite_Key_Out,Voucher Number_cus,My GSTIN_cus,Supplier GSTIN_cus,Item Taxable Value *_cus,Composite_Key_Cus
str,i64,str,str,f64,str,i64,str,str,f64,str
"""423110034326AA…",4231100343,"""26AADCK8554D1Z…","""27AACFE0874A1Z…",179200.0,"""423110034326AA…",4231100343,"""26AADCK8554D1Z…","""27AACFE0874A1Z…",179200.0,"""423110034326AA…"
"""423920004926AA…",4239200049,"""26AADCK8554D1Z…",null,6000.0,"""423920004926AA…",4239200049,"""26AADCK8554D1Z…",null,6000.0,"""423920004926AA…"
"""423140000126AA…",4231400001,"""26AADCK8554D1Z…","""29AAPFB4767N2Z…",1150.0,"""423140000126AA…",4231400001,"""26AADCK8554D1Z…","""29AAPFB4767N2Z…",1150.0,"""423140000126AA…"
"""423110039126AA…",4231100391,"""26AADCK8554D1Z…","""24AAACB3333J1Z…",1.33e6,"""423110039126AA…",4231100391,"""26AADCK8554D1Z…","""24AAACB3333J1Z…",1.33e6,"""423110039126AA…"
"""623460007727AA…",6234600077,"""27AADCK8554D4Z…",null,24320.0,"""623460007727AA…",6234600077,"""27AADCK8554D4Z…",null,24320.0,"""623460007727AA…"
"""423220012026AA…",4232200120,"""26AADCK8554D1Z…","""24AAVFR2508P1Z…",32214.0,"""423220012026AA…",4232200120,"""26AADCK8554D1Z…","""24AAVFR2508P1Z…",32214.0,"""423220012026AA…"
"""423180001426AA…",4231800014,"""26AADCK8554D1Z…","""24AHPPM8600A1Z…",1179.0,"""423180001426AA…",4231800014,"""26AADCK8554D1Z…","""24AHPPM8600A1Z…",1179.0,"""423180001426AA…"
"""423250004426AA…",4232500044,"""26AADCK8554D1Z…","""27AAACU1299G1Z…",2370.0,"""423250004426AA…",4232500044,"""26AADCK8554D1Z…","""27AAACU1299G1Z…",2370.0,"""423250004426AA…"
"""423110015726AA…",4231100157,"""26AADCK8554D1Z…","""27AACCL2337A1Z…",168300.0,"""423110015726AA…",4231100157,"""26AADCK8554D1Z…","""27AACCL2337A1Z…",168300.0,"""423110015726AA…"
